@origin script link: [Keywords Tagging](https://aiprd-jupyter.cisco.com/user/hdpiopsdp/notebooks/HadoopDir/ai_xray/scripts/ds/related_bugs/Keywords%20Tagging.ipynb#Tags-Generation) 

@ text annotation tool: [Doccano](https://github.com/doccano/doccano)

In [5]:
import pandas as pd
import spacy
import random
import sys
import json
from spacy import displacy

# Training the NER model

convert doccano exported json file to format that spcay can digest

In [2]:
# JSON(Text-Labels) file exported from doccano

fo = open('/survey_annotated_data.json', "r")
lines = fo.readlines()

TRAIN_DATA = []
LABELS = ['Keywords', 'Products', 'Counterparty', 'Positive', 'Negative']

for line in lines:

    line = json.loads(line)
    if "labels" in line:
        line["entities"] = line.pop("labels")
    else:
        line["entities"] = []

    tmp_ents = []
    for e in line["entities"]:
        
        if e[2] in LABELS:
            tmp_ents.append(tuple([e[0], e[1], e[2]]))
        
        line["entities"] = tmp_ents

    if (len(line["text"]) > 5):
        
        #print(tuple([line["text"],  {'entities':tmp_ents}]))
        TRAIN_DATA.append(tuple([line["text"],  {'entities':tmp_ents}]))


In [11]:
#TRAIN_DATA[:6]

In [4]:
# format after conversion：
# TRAIN_DATA = [('what is the price of polo?', {'entities': [(21, 25, 'PrdName')]}), 
#               ('what is the price of ball?', {'entities': [(21, 25, 'PrdName')]}), 
#               ('what is the price of jegging?', {'entities': [(21, 28, 'PrdName')]}), 


TRAINING_ITERATION = 5
TRAINING_DATA = TRAIN_DATA

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    #nlp = spacy.load('en')  # load Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAINING_DATA, TRAINING_ITERATION)

/home/jupyter/.local/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs


Statring iteration 0


/home/jupyter/.local/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text ""agree with all above.  Assurance: the data is ina..." with entities "[(24, 33, 'Products'), (120, 128, 'Products'), (22...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/home/jupyter/.local/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "1. quality is an issue.  2. the. information is to..." with entities "[(141, 147, 'Counterparty'), (3, 10, 'Keywords'), ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/home/jupyter/.local/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some

{'ner': 1090.410192590301}
Statring iteration 1
{'ner': 394.40711802133103}
Statring iteration 2
{'ner': 319.61087659593807}
Statring iteration 3
{'ner': 600.141592655642}
Statring iteration 4
{'ner': 274.4559846504437}


In [23]:
# sasve the model to disk
#prdnlp.to_disk(PATH+'survey_spacy_ner_model')

# Visualizing the keywords

In [24]:
text = "But dakan wei Google PS4 is starting from behind. dakan wei still like playing Nintendo Switch, The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."

doc = prdnlp(text)
#print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Google PS4 14 24 Keywords
behind 42 48 Keywords
dakan 50 55 Keywords
Switch 88 94 Counterparty
late push 115 124 Keywords
Apple 144 149 Keywords
Amazon 184 190 Counterparty
Alexa software 193 207 Keywords
on 220 222 Counterparty
Echo 227 231 Keywords
Dot 236 239 Counterparty
clear leads 254 265 Keywords
adoption 278 286 Keywords


In [27]:
prdnlp = spacy.load(PATH + "survey_spacy_ner_model")

Input text data:

In [12]:
text = "But dakan wei Google PS4 is starting from behind. dakan wei still like playing Nintendo Switch, The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."


In [13]:
# https://spacy.io/usage/visualizers#ent
# colors = {"KEYWORDS": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}

colors = {"KEYWORDS":     "#D8E0BB",
         "PRODUCTS":      "#35BCEC",
         "COUNTERPARTY":  "#F6D55C",
         "POSITIVE":      "#4BFA6B",
         "NEGATIVE":      "#FA2A24"}


options = {"colors": colors}


doc = prdnlp(text)
displacy.render(doc, style="ent", jupyter=True, options=options)
